# **AI Candidate Screening Agent using Amazon Bedrock AgentCore Runtime, CrewAI and Gemini 2.5 Flash**

This agent help you analyze your CV against the job requirements, provide result for the next recruitment step, create rejection/interview email and create interview question.

In [1]:
import os
# Retrieve AWS credentials from Colab Secrets
from google.colab import userdata
os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWSACCESSKEY')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWSSECRETKEY')
# Create the folder
folder_name = "runtime"
os.makedirs(folder_name, exist_ok=True)

In [2]:
%%writefile runtime/requirements.txt
bedrock-agentcore-starter-toolkit
bedrock-agentcore
crewai
google-genai
pypdf
boto3
pydantic

Writing runtime/requirements.txt


In [ ]:
!pip install -r runtime/requirements.txt

In [4]:
%%writefile runtime/crewai_agentcore.py
from crewai import LLM, Agent, Task, Crew, Process
from crewai.tools import tool
from pydantic import BaseModel, Field
from typing import Optional
import boto3
import datetime
import json
import random
import os
import tempfile

# Exclusive deploy to AgentCore Runtime
from bedrock_agentcore.runtime import BedrockAgentCoreApp
app = BedrockAgentCoreApp()

region = "us-west-2"
s3 = boto3.client('s3', region_name=region)

# Retrieve Gemini API Key from AWS Secret Manager
secretmanager = boto3.client('secretsmanager', region_name="us-west-2")
response = secretmanager.get_secret_value(SecretId='geminiapikey')
secret_json = json.loads(response["SecretString"])
api_key = secret_json["GEMINI_API_KEY"]
llm = LLM(model="gemini/gemini-2.5-flash", api_key=api_key, temperature=0.1)

# Structure of this AI Agent
# User --> Extract CV --> Compare and Match between Job Requirements and CV --> Result to the next step of the recruitment process -->
# Create Interview Email --> Create Interview Question --> End OR Create Rejection Email -> End

# Agent 1: Compare and Match between Job Requirements and CV
class compareMatchClass(BaseModel):
    """Compare and match between job requirements and curriculum vitae."""
    minimal_requirements_analysis: str = Field(description="Analysis of minimal requirements")
    preferred_requirements_analysis: str = Field(description="Analysis of preferred requirements")
    strengths: str = Field(description="Strengths of candidate")
    potential_gaps: str = Field(description="Potential gaps of candidate")
    candidate_name: str = Field(description="Candidate name in the CV")

compareAndMatchAgent = Agent(
    role="Compare and Match Agent",
    goal="Compare and match between job requirements and extracted of curriculum vitae",
    backstory="""
    You are an expert HR recruiter with a keen eye for detail. Your primary function is to compare and match a candidate curriculum vitae (CV) against job requirements.
    Your insights are crucial for determining a candidate's suitability for the next stages of the recruitment process.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=compareMatchClass
)

compareAndMatchTask = Task(
    description="""
    You are a virtual human resources expert. You help me COMPARE and MATCH between {cv_data} and AI engineer position job requirements like this :
    1. Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.
    2. Fine-tune and integrate LLMs (like OpenAI and Gemini) into ERP workflows.
    3. Build smart features such as recommendation engines, forecasting modules, NLP tools, and more.
    Preferred requirements :
    Develop and maintain scalable cloud-based AI solutions across multi-cloud platforms (AWS, GCP, Azure).
    """,
    expected_output="""
    Minimal Requirements Analysis :

    Requirement 1:

    Requirement n (n is the number of requirement):

    Preferred Requirement Analysis :

    Strengths:

    Potential Gaps:

    Candidate Name:
    """,
    agent=compareAndMatchAgent
)

# Agent 2: Score to the Next Step of the Recruitment Process
class scoreNextStepClass(BaseModel):
    """Score value to proceed to the next step in the recruitment process."""
    score: int = Field(description="Score of candidate matched")

scoreNextStepAgent = Agent(
    role="Score to Next Step of the Recruitment Process Agent",
    goal="Provide score to next step of the recruitment process",
    backstory="""
    You are an expert in HR recruitment processes, capable of assessing candidate suitability and determining the next logical step in their application journey.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=scoreNextStepClass
)

scoreNextStepTask = Task(
    description="""
    You are a virtual human resources expert. You help me add value from 0 until 10 to the next step of recruitment process.
    Answer with a number between 0 and 10 ONLY without any additions.
    """,
    expected_output="""
    1
    """,
    agent=scoreNextStepAgent
)

# Agent 3: Create Rejection/Interview Email
class createEmailClass(BaseModel):
    """Create rejection email or interview invitation email for candidate."""
    email: str = Field(description="Body of email for candidate. Use `[CANDIDATE_NAME]`, `[INTERVIEW_DATE]` and `[INTERVIEW_TIME]` as placeholders for interview emails.")

createEmailAgent = Agent(
    role="Create Rejection or Interview Email Agent",
    goal="Crate rejection or interview email to next step of the recruitment process",
    backstory="""
    You are an expert in HR recruitment processes, capable of writing rejection or interview email for candidate.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=createEmailClass
)

createEmailTask = Task(
    description="""
    You are a virtual human resources expert. You are help me create email in the recruitment process.

    If the score is LESS THAN 7 out of 10, create a very simple rejection about failed to AI engineer position email for unsuccessful candidate.
    Write output structure like this:
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Thanks,\n
    HRD of AgentCore.

    But if the score is MORE THAN 7 out of 10, create an interview email for a candidate accepted to the next step to AI engineer position. Write output structure like this:
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Date : {interview_date}\n
    Time : {interview_time}\n
    Google Meet interview link : https://bit.ly/agentcore-interview\n
    ........\n
    Thanks,\n
    HRD of AgentCore.
    """,
    expected_output="""
    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Thanks,\n
    HRD of AgentCore.

    OR

    Hello, {compareAndMatchTask.output.candidate_name}\n
    ........\n
    Date : {interview_date}\n
    Time : {interview_time}\n
    Google Meet interview link : https://bit.ly/agentcore-interview\n
    ........\n
    Thanks,\n
    HRD of AgentCore.
    """,
    agent=createEmailAgent
)

def createEmail():
    # Formatted interview date (today + 3 days)
    today = datetime.date.today()
    interview_date = today + datetime.timedelta(days=3)
    formatted_date = interview_date.strftime("%d-%m-%Y")

    # Formatted interview time between 1 PM (13:00) and 4 PM (16:00)
    hour = random.randint(13, 16)
    minute = random.choice([0, 30])
    if hour == 16 and minute == 30:
        minute = 0 # if 4:30 PM was chosen, adjust to 4:00 PM
    interview_time = datetime.time(hour, minute, 0)
    formatted_time = interview_time.strftime("%I:%M %p") # Format to HH:MM AM/PM
    return formatted_date, formatted_time
formatted_date, formatted_time = createEmail()

# Agent 4: Create Interview Questions
class createInterviewQuestionClass(BaseModel):
    """Create interview questions for candidates who are accepted to the next step."""
    questions: str = Field(description="Interview questions for candidate")

createInterviewQuestionAgent = Agent(
    role="Create Interview Question Agent",
    goal="Create interview question for interview session of the recruitment process",
    backstory="""
    You are an experienced HR interviewer, skilled at formulating insightful and relevant questions.
    Your objective is to design interview questions that effectively probe a candidate's skills, experience, and suitability based on their curriculum vitae.
    """,
    verbose=True,
    allow_delegation=False,
    memory=True,
    llm=llm,
    output_pydantic=createInterviewQuestionClass
)

createInterviewQuestionTask = Task(
    description="""
    You are a virtual human resources expert. You are help me create 3 interview questions about CV PDF file that already extracted.

    If the score is LESS THAN 7 out of 10, DO NOT create interview questions. Write output structure like this:
    {\"questions\": \"-\"}

    But if the score is MORE THAN 7 out of 10, create 3 interview questions about CV PDF file that already extracted. Write output structure like this:
    QUESTION 1 : ........ \n
    QUESTION 2 : ........ \n
    QUESTION 3 : ........ \n
    """,
    expected_output="""
    {\"questions\": \"-\"}

    OR

    QUESTION 1 : ........ \n
    QUESTION 2 : ........ \n
    QUESTION 3 : ........ \n
    """,
    agent=createInterviewQuestionAgent
)

# Create AI agent crew
candidate_screening_crew = Crew(
    agents=[compareAndMatchAgent, scoreNextStepAgent, createEmailAgent, createInterviewQuestionAgent],
    tasks=[compareAndMatchTask, scoreNextStepTask, createEmailTask, createInterviewQuestionTask],
    process=Process.sequential,
    verbose=True,
)

# Extract CV PDF file
def extract_cv(inputpdf):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_pdf_file:
        local_file_path = temp_pdf_file.name
        s3.download_file("screening-candidate", inputpdf, local_file_path)
    from pypdf import PdfReader
    reader = PdfReader(local_file_path)
    page = reader.pages[0]
    text = page.extract_text()
    return text
    os.remove(local_file_path)

@app.entrypoint
def crewai_agentcore(payload):
    # This input is the filename of the PDF
    pdf_filename = payload.get("inputpdf")

    # Extract text from the PDF file
    user_input = extract_cv(pdf_filename)
    result = candidate_screening_crew.kickoff(inputs={
        'cv_data': user_input,
        'interview_date': formatted_date,
        'interview_time': formatted_time
    })

    # Output from Agent 1
    camt = compareAndMatchTask.output.raw

    # Output from Agent 2
    snst = scoreNextStepTask.output.raw

    # Output from Agent 3
    cet = createEmailTask.output.raw

    # Output from Agent 4
    rraw = result.raw

    # Delete CV PDF file in S3 after processing
    s3.delete_object(Bucket="screening-candidate", Key=pdf_filename)
    return camt, snst, cet, rraw

app.run()

Writing runtime/crewai_agentcore.py


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()
region="us-west-2"
agent_name = "gemini_crewai"

response = agentcore_runtime.configure(
    entrypoint="runtime/crewai_agentcore.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="runtime/requirements.txt",
    region=region,
    agent_name=agent_name
)
response

In [6]:
print("AgentCore Runtime is configured.")

AgentCore Runtime is configured.


In [ ]:
launch_result = agentcore_runtime.launch()

# **Temporary Stop! Your AgentCore Runtime now available but don't invoke directly because S3 bucket access and Gemini API Key in AWS Secret Manager not yet added to your IAM role for AgentCore Runtime**

**1. Go to AWS Secret Manager, click secret name then copy Secret ARN of Gemini API Key.**

**2. Go to Amazon Bedrock AgentCore -> Agent runtime then click your agent name that already created. Click "Version 1" then click IAM service role of Permissions that automatically opened a new tab (e.g. AmazonBedrockAgentCoreSDKRuntime-{region-name}-{random-number-letter}).**

**3. Click IAM policy name that related (e.g. BedrockAgentCoreRuntimeExecutionPolicy-{your-agent-name}).**

**4. Add your Secret ARN of Gemini API Key in resource of "secretsManager:GetSecretValue" action then add service (S3) and add resource then click Next then click Save. DONE**

# **Continue to invoke your AgentCore Runtime now**

# **Execute First Curriculum Vitae**

In [8]:
import boto3
import json
s3 = boto3.client('s3', 'us-west-2')

def upload_to_s3(file_path):
    s3.upload_file(file_path, "screening-candidate", file_path)
    return file_path

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client('bedrock-agentcore', "us-west-2")

In [10]:
pdf = "Always Winner CV.pdf"
cvpdf = upload_to_s3(pdf)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    payload=json.dumps({"inputpdf": cvpdf})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
print(response_data)

['Minimal Requirements Analysis :\n\nRequirement 1: Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.\nMatch: The candidate explicitly lists "MLOps" under technical skills, which directly encompasses the entire AI lifecycle. "Data Engineering" is listed, covering data preparation. "Machine Learning" skills such as Supervised Learning, Unsupervised Learning, Transfer Learning, Model Optimization, and Hyperparameter Tuning demonstrate proficiency in model development and evaluation. Furthermore, tools like "Docker, Kubernetes, Git, MLflow, Weights & Biases" are crucial for deployment and managing the lifecycle, all of which are listed. "API Development" also supports deployment.\n\nRequirement 2: Fine-tune and integrate LLMs (like OpenAI and Gemini) into ERP workflows.\nMatch: The candidate possesses strong expertise in "Generative AI," specifically listing "LLMs (GPT, BERT, LLaMA)" and "Hugging Face Transformers." This indicates a so

In [11]:
print(response_data[0])

print(response_data[1])

print(response_data[2])

print(response_data[3])

Minimal Requirements Analysis :

Requirement 1: Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.
Match: The candidate explicitly lists "MLOps" under technical skills, which directly encompasses the entire AI lifecycle. "Data Engineering" is listed, covering data preparation. "Machine Learning" skills such as Supervised Learning, Unsupervised Learning, Transfer Learning, Model Optimization, and Hyperparameter Tuning demonstrate proficiency in model development and evaluation. Furthermore, tools like "Docker, Kubernetes, Git, MLflow, Weights & Biases" are crucial for deployment and managing the lifecycle, all of which are listed. "API Development" also supports deployment.

Requirement 2: Fine-tune and integrate LLMs (like OpenAI and Gemini) into ERP workflows.
Match: The candidate possesses strong expertise in "Generative AI," specifically listing "LLMs (GPT, BERT, LLaMA)" and "Hugging Face Transformers." This indicates a solid foun

# **Execute Second Curriculum Vitae**

In [12]:
pdf = "Sonny Wawwak CV.pdf"
cvpdf = upload_to_s3(pdf)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    payload=json.dumps({"inputpdf": cvpdf})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
print(response_data)

["Minimal Requirements Analysis :\n\nRequirement 1:\nAnalysis: The candidate, Sonny Wawwak, is a UI/UX Designer with experience in design-centric tasks such as wireframing, prototyping, user flows, and usability testing. Their CV does not indicate any experience or skills related to data preparation, AI model development, evaluation, or deployment. The professional experience is solely in UI/UX design roles.\nMatch: No match.\n\nRequirement 2:\nAnalysis: Sonny Wawwak's skills list includes UI Design, UX Research, and various design tools. There is no mention of Large Language Models (LLMs) such as OpenAI or Gemini, nor any experience with ERP systems or the integration of advanced AI models.\nMatch: No match.\n\nRequirement 3:\nAnalysis: The candidate's experience is focused on creating user interfaces and user experiences. There is no evidence in the CV of skills or experience in developing complex AI-driven features like recommendation engines, forecasting modules, or Natural Languag

In [13]:
print(response_data[0])

print(response_data[1])

print(response_data[2])

print(response_data[3])

Minimal Requirements Analysis :

Requirement 1:
Analysis: The candidate, Sonny Wawwak, is a UI/UX Designer with experience in design-centric tasks such as wireframing, prototyping, user flows, and usability testing. Their CV does not indicate any experience or skills related to data preparation, AI model development, evaluation, or deployment. The professional experience is solely in UI/UX design roles.
Match: No match.

Requirement 2:
Analysis: Sonny Wawwak's skills list includes UI Design, UX Research, and various design tools. There is no mention of Large Language Models (LLMs) such as OpenAI or Gemini, nor any experience with ERP systems or the integration of advanced AI models.
Match: No match.

Requirement 3:
Analysis: The candidate's experience is focused on creating user interfaces and user experiences. There is no evidence in the CV of skills or experience in developing complex AI-driven features like recommendation engines, forecasting modules, or Natural Language Processing 